In [1]:
import pandas as pd
import numpy as np
import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
f_path = "C:/Users/Dell/Documents/data/FQ304大修人员剂量明细.xlsx"

In [3]:
# 确定数据源和数据类型
data = pd.read_excel(
    f_path,
    sheet_name="数据源",
    usecols=[
        "人员编号",
        "姓名",
        "单位",
        "处室",
        "机组",
        "进入区域",
        "进入时间",
        "离开时间",
        "持续时间(h)",
        "EPD-γ剂量(mSv)"
    ],
    dtype={
        "人员编号": str,
        "进入时间": "datetime64",
        "离开时间": "datetime64"
    }
)

In [4]:
data.head()

,姓名,人员编号,单位,处室,机组,进入区域,进入时间,离开时间,持续时间(h),EPD-γ剂量(mSv)
0,尚子忠,80204320,浙江创想,维修支持处,福一厂3号机组,8LX,2021-08-28 15:40:06,2021-08-28 18:44:39,3.08,0.579
1,尚子忠,80204320,浙江创想,维修支持处,福一厂3号机组,8LX,2021-09-06 18:55:58,2021-09-06 20:46:39,1.84,0.551
2,李永松,80195004,上海申亿,维修支持处,福一厂3号机组,8LX,2021-09-09 13:17:09,2021-09-09 17:34:53,4.30,0.550
3,曹勇华,80174391,浙江创想,维修支持处,福一厂3号机组,8LX,2021-08-28 15:39:38,2021-08-28 19:34:28,3.91,0.540
4,陈文清,80162361,上海申亿,维修支持处,福一厂3号机组,8LX,2021-09-09 13:16:41,2021-09-09 17:35:03,4.31,0.538


In [26]:
cut_bins = [-np.infty, 0.5, 1.0, 2.0, 5.0, np.infty]
bins_labels = ["0-0.5", "0.5-1.0", "1.0-2.0", "2.0-5.0", ">5.0"]
df=pd.cut(data["EPD-γ剂量(mSv)"], bins=cut_bins, labels=bins_labels)
df.value_counts()

0-0.5      26213
0.5-1.0        6
1.0-2.0        0
2.0-5.0        0
>5.0           0
Name: EPD-γ剂量(mSv), dtype: int64

In [18]:
date = "2021-09-01"
df = data.query("离开时间.dt.date.astype('str') == @date").head(50)

pvt_dt = pd.pivot_table(
    df,
    index="单位",
    # values="单位",
    aggfunc={"单位": "count"}
)
pvt_dt

,单位
单位,
上海申亿,2
中核检修有限公司,35
中核武汉核电运行技术股份有限公司,11
中辐院,2


In [ ]:
# 设置要筛选的数据源的日期范围
# 设定起止日期时间
# 实际结束时间为2021/9/19 11:46
first_datetime = pd.to_datetime("2021/8/22 8:00")
last_datetime = pd.to_datetime("2021/9/20 00:00")
# 设置若在日期范围内，则以昨天为范围的终止日期，否则以截止日期范围的终止日期
now = datetime.datetime.now()
if now <= last_datetime:
    end_datetime = now - datetime.timedelta\
        (hours=now.hour, minutes=now.minute, seconds=now.second)
else:
    end_datetime = last_datetime

start_datetime = end_datetime - datetime.timedelta(days = 1)

In [ ]:
data.columns

In [ ]:
data.dtypes

In [ ]:
# 筛选日期范围内的数据
queried_data = data.query("离开时间 >= @first_datetime and 离开时间 <= @end_datetime")
queried_data.head()

In [ ]:
queried_data.info()

In [ ]:
# 透视汇总筛选的数据，以离开时间、人员编号、姓名、处室、单位为索引
# 统计每日每人的累计剂量、人次和工作时间
person_total_dose = pd.pivot_table(
    queried_data,
    # values=["EPD-γ剂量(mSv)", "人员编号", "持续时间(h)"],
    index=[
        # 将离开时间以日为单位分组
        pd.Grouper(key="离开时间", freq="D"),
        "人员编号",
        "姓名",
        "处室",
        "单位"
    ],
    aggfunc={
        "EPD-γ剂量(mSv)": np.sum,
        "人员编号": np.count_nonzero,
        "持续时间(h)": np.sum
    },
    # margins=True, margins_name="总计"
)\
    # .stack(0).reset_index(0)
person_total_dose.head(10)

In [ ]:
# 将每天的人员数据按照剂量降序排序
person_total_dose.sort_values(by = "EPD-γ剂量(mSv)", ascending=False)\
    .sort_index(level = 0, sort_remaining=False)

In [ ]:
# 选择某一天的第50名的信息
person_total_dose.loc[("2021-08-22",slice(None)),:].iloc[49,:]\
# .name[4]

In [ ]:
# 统计每日的累计剂量、人次和工作时间
day_total_data = queried_data.groupby(
    pd.Grouper(key="离开时间", freq="D"),
).agg(
    {"EPD-γ剂量(mSv)": np.sum,
    "人员编号": np.count_nonzero,
    "持续时间(h)": np.sum
}
)
day_total_data

In [ ]:
# 每日剂量累加
day_total_dose = day_total_data["EPD-γ剂量(mSv)"].cumsum()
day_total_dose

In [ ]:
fig1 = make_subplots(specs=[[{"secondary_y": True}]])
fig1.add_trace(
    go.Bar(
        x = day_total_data.index,
        y = day_total_data["EPD-γ剂量(mSv)"],
        name = "每日剂量"
    ),
    secondary_y=False
)

fig1.add_trace(
    go.Scatter(
        x = day_total_dose.index,
        y = day_total_dose,
        name = "累计剂量"
    ),
    secondary_y=True
)

fig1.update_layout(
    title_text = "每日剂量和累计剂量"
)

fig1.update_xaxes(
    tickformat = "%d日\n%Y年%m月"
)

fig1.update_yaxes(
    title_text = "每日剂量(mSv)", secondary_y=False
)
fig1.update_yaxes(
    title_text = "累计剂量(mSv)", secondary_y=True
)

In [ ]:
# 每日人次累加
day_total_person = day_total_data["人员编号"].cumsum()
day_total_person

In [ ]:
fig2 = make_subplots(specs=[[{"secondary_y": True}]])
fig2.add_trace(
    go.Bar(
        x = day_total_data.index,
        y = day_total_data["人员编号"],
        name = "每日人次"
    ),
    secondary_y=False
)

fig2.add_trace(
    go.Scatter(
        x = day_total_person.index,
        y = day_total_person,
        name = "累计人次"
    ),
    secondary_y=True
)

fig2.update_layout(
    title_text = "每日人次和累计人次"
)

fig2.update_xaxes(
    tickformat = "%d日\n%Y年%m月"
)

fig2.update_yaxes(
    title_text = "每日人次", secondary_y=False
)
fig2.update_yaxes(
    title_text = "累计人次", secondary_y=True
)